# Manejo de faltantes

In [1]:
import pandas as pd
import numpy as np

Intel MKL WARNING: Support of Intel(R) Streaming SIMD Extensions 4.2 (Intel(R) SSE4.2) enabled only processors has been deprecated. Intel oneAPI Math Kernel Library 2025.0 will require Intel(R) Advanced Vector Extensions (Intel(R) AVX) instructions.
Intel MKL WARNING: Support of Intel(R) Streaming SIMD Extensions 4.2 (Intel(R) SSE4.2) enabled only processors has been deprecated. Intel oneAPI Math Kernel Library 2025.0 will require Intel(R) Advanced Vector Extensions (Intel(R) AVX) instructions.



```python
from google.colab import drive

drive.mount('/content/gdrive')

path = "/content/gdrive/MyDrive/ucema2023/clases/clase_8/NFL Play by Play 2009-2016 (v3).csv"
nfl_data = pd.read_csv(path)

crear una semilla

np.random.seed(0)


___

Vamos a hacer inspeccion basica del dataset y buscar sus faltantes


In [2]:
path = "/Users/lolonastri/Desktop/POO_UCEMA/machine_learning/3-manejo_faltantes/teoria/nfl.csv"
nfl_data = pd.read_csv(path)

# crear una semilla
np.random.seed(0)

/var/folders/xx/hcklcr8s0rxd7p5fz63z342w0000gn/T/ipykernel_36359/3785920783.py:2: DtypeWarning: Columns (25,51) have mixed types. Specify dtype option on import or set low_memory=False.
  nfl_data = pd.read_csv(path)


In [3]:
nfl_data.head()

,Date,GameID,Drive,qtr,down,time,TimeUnder,TimeSecs,PlayTimeDiff,SideofField,...,yacEPA,Home_WP_pre,Away_WP_pre,Home_WP_post,Away_WP_post,Win_Prob,WPA,airWPA,yacWPA,Season
0,2009-09-10,2009091000,1,1,NaN,15:00,15,3600.0,0.0,TEN,...,NaN,0.485675,0.514325,0.546433,0.453567,0.485675,0.060758,NaN,NaN,2009
1,2009-09-10,2009091000,1,1,1.0,14:53,15,3593.0,7.0,PIT,...,1.146076,0.546433,0.453567,0.551088,0.448912,0.546433,0.004655,-0.032244,0.036899,2009
2,2009-09-10,2009091000,1,1,2.0,14:16,15,3556.0,37.0,PIT,...,NaN,0.551088,0.448912,0.510793,0.489207,0.551088,-0.040295,NaN,NaN,2009
3,2009-09-10,2009091000,1,1,3.0,13:35,14,3515.0,41.0,PIT,...,-5.031425,0.510793,0.489207,0.461217,0.538783,0.510793,-0.049576,0.106663,-0.156239,2009
4,2009-09-10,2009091000,1,1,4.0,13:27,14,3507.0,8.0,PIT,...,NaN,0.461217,0.538783,0.558929,0.441071,0.461217,0.097712,NaN,NaN,2009



Sí, parece que faltan algunos valores.

___

# ¿Cuántos puntos de datos faltantes tenemos?

Bien, ahora sabemos que nos faltan algunos valores. Veamos cuántos tenemos en cada columna.

In [4]:
missing_values_count = nfl_data.isnull().sum()

# ahora cuantos tenemos en las primeras 10 columnas
missing_values_count[0:10]

Date                0
GameID              0
Drive               0
qtr                 0
down            54218
time              188
TimeUnder           0
TimeSecs          188
PlayTimeDiff      374
SideofField       450
dtype: int64

Es clave tener una referencia a nivel de cuanto es el porcentual de esos faltantes

In [5]:
total_cells = np.product(nfl_data.shape) # total de celdas (filas * columnas)
total_missing = missing_values_count.sum()

# porcentaje de datos
percent_missing = (total_missing/total_cells) * 100
print(percent_missing)

27.652267428200588


¡Vaya, casi una cuarta parte de las celdas de este conjunto de datos están vacías! En el siguiente paso, veremos más de cerca algunas de las columnas con valores faltantes y trataremos de descubrir qué podría estar pasando con ellas.

___ 
# Descubra por qué faltan los datos

Este es el punto en el que entramos en la parte de la ciencia de datos que me gusta llamar "intución de datos", con lo que me refiero a "mirar realmente los datos y tratar de descubrir por qué son como son y cómo serán". afectar su análisis". Puede ser una parte frustrante de la ciencia de datos, especialmente si eres nuevo en el campo y no tienes mucha experiencia. Para lidiar con valores faltantes, necesitarás usar tu intuición para descubrir por qué falta el valor. Una de las preguntas más importantes que puedes hacerte para ayudarte a resolver esto es la siguiente:

> **¿Falta este valor porque no se registró o porque no existe?**

Si falta un valor porque no existe (como la altura del hijo mayor de alguien que no tiene hijos), entonces no tiene sentido intentar adivinar cuál podría ser. Probablemente desee conservar estos valores como "NaN". Por otro lado, si falta un valor porque no se registró, puede intentar adivinar cuál podría haber sido basándose en los otros valores de esa columna y fila. Esto se llama **imputación** y ¡aprenderemos cómo hacerlo a continuación! :)

Trabajemos con un ejemplo. Al observar la cantidad de valores faltantes en el marco de datos `nfl_data`, noto que la columna "TimesSec" tiene muchos valores faltantes:

In [6]:
missing_values_count[0:10]

Date                0
GameID              0
Drive               0
qtr                 0
down            54218
time              188
TimeUnder           0
TimeSecs          188
PlayTimeDiff      374
SideofField       450
dtype: int64

Al mirar la documentacion, puedo ver que esta columna tiene información sobre la cantidad de segundos que quedaban en el juego cuando se realizó la jugada. Esto significa que estos valores probablemente faltan porque no se registraron y no porque no existen. Por lo tanto, tendría sentido que intentáramos adivinar cuáles deberían ser en lugar de simplemente dejarlos como de NA.

Por otro lado, hay otros campos, como "PenalizedTeam" a los que también les faltan muchos registros. En este caso, sin embargo, falta el campo porque si no hubo penalización entonces no tiene sentido decir *qué* equipo fue penalizado. Para esta columna, tendría más sentido dejarla vacía o agregar un tercer valor como "ninguno" y usarlo para reemplazar los NA.

> **Consejo:** ¡Este es un excelente lugar para leer la documentación del conjunto de datos si aún no lo has hecho! Si está trabajando con un conjunto de datos que obtuvo de otra persona, también puede intentar comunicarse con ella para obtener más información.

Si está haciendo un análisis de datos muy cuidadoso, este es el punto en el que mirará cada columna individualmente para descubrir la mejor estrategia para completar esos valores faltantes. En el resto de este cuaderno, cubriremos algunas técnicas "rápidas y sucias" que pueden ayudarle con los valores faltantes, pero que probablemente también terminarán eliminando información útil o agregando algo de ruido a sus datos.

___ 
# Elimina los valores faltantes

Si tiene prisa o no tiene una razón para descubrir por qué faltan valores, una opción que tiene es simplemente eliminar las filas o columnas que contengan valores faltantes. (Nota: ¡generalmente no recomiendo este enfoque para proyectos importantes! Por lo general, vale la pena tomarse el tiempo para revisar sus datos y observar realmente todas las columnas con valores faltantes, una por una, para conocer realmente su conjunto de datos. .)

Si está seguro de que desea eliminar filas con valores faltantes, pandas tiene una función útil, `dropna()` para ayudarle a hacerlo. ¡Probémoslo en nuestro conjunto de datos de la NFL!

In [7]:
# borramos todos
nfl_data.dropna()

,Date,GameID,Drive,qtr,down,time,TimeUnder,TimeSecs,PlayTimeDiff,SideofField,...,yacEPA,Home_WP_pre,Away_WP_pre,Home_WP_post,Away_WP_post,Win_Prob,WPA,airWPA,yacWPA,Season


¡Dios mío, parece que eso ha eliminado todos nuestros datos! 😱 Esto se debe a que a cada fila de nuestro conjunto de datos le faltaba al menos un valor. Es posible que tengamos más suerte eliminando todas las *columnas* a las que les falta al menos un valor.

In [8]:
columns_with_na_dropped = nfl_data.dropna(axis=1)
columns_with_na_dropped.head()

,Date,GameID,Drive,qtr,TimeUnder,ydstogo,ydsnet,PlayAttempted,Yards.Gained,sp,...,AwayTeam,Timeout_Indicator,posteam_timeouts_pre,HomeTimeouts_Remaining_Pre,AwayTimeouts_Remaining_Pre,HomeTimeouts_Remaining_Post,AwayTimeouts_Remaining_Post,ExPoint_Prob,TwoPoint_Prob,Season
0,2009-09-10,2009091000,1,1,15,0,0,1,39,0,...,TEN,0,3,3,3,3,3,0.0,0.0,2009
1,2009-09-10,2009091000,1,1,15,10,5,1,5,0,...,TEN,0,3,3,3,3,3,0.0,0.0,2009
2,2009-09-10,2009091000,1,1,15,5,2,1,-3,0,...,TEN,0,3,3,3,3,3,0.0,0.0,2009
3,2009-09-10,2009091000,1,1,14,8,2,1,0,0,...,TEN,0,3,3,3,3,3,0.0,0.0,2009
4,2009-09-10,2009091000,1,1,14,8,2,1,0,0,...,TEN,0,3,3,3,3,3,0.0,0.0,2009


In [9]:
# cuantos datos perdimos
print("Columnas en dataset original: %d \n" % nfl_data.shape[1])
print("Columnas con borradas na's: %d" % columns_with_na_dropped.shape[1])

Columnas en dataset original: 102 

Columnas con borradas na's: 37


Hemos perdido bastantes datos, pero en este punto hemos eliminado con éxito todos los "NaN" de nuestros datos.

___

# Completar los valores faltantes automáticamente

Otra opción es intentar completar los valores que faltan. Para la siguiente parte, obtendré una pequeña subsección de datos de la NFL para que se imprima bien.

In [10]:
# obtener un subset de datos del general
subset_nfl_data = nfl_data.loc[:, 'EPA':'Season'].head() # me trae todas las columnas que hay entre EPA y Season y me saca el head de eso.
subset_nfl_data

,EPA,airEPA,yacEPA,Home_WP_pre,Away_WP_pre,Home_WP_post,Away_WP_post,Win_Prob,WPA,airWPA,yacWPA,Season
0,2.014474,NaN,NaN,0.485675,0.514325,0.546433,0.453567,0.485675,0.060758,NaN,NaN,2009
1,0.077907,-1.068169,1.146076,0.546433,0.453567,0.551088,0.448912,0.546433,0.004655,-0.032244,0.036899,2009
2,-1.402760,NaN,NaN,0.551088,0.448912,0.510793,0.489207,0.551088,-0.040295,NaN,NaN,2009
3,-1.712583,3.318841,-5.031425,0.510793,0.489207,0.461217,0.538783,0.510793,-0.049576,0.106663,-0.156239,2009
4,2.097796,NaN,NaN,0.461217,0.538783,0.558929,0.441071,0.461217,0.097712,NaN,NaN,2009


Podemos usar la función `fillna()` de Pandas para completar los valores faltantes en un marco de datos. Una opción que tenemos es especificar con qué queremos que se reemplacen los valores `NaN`. Aquí, estoy diciendo que me gustaría reemplazar todos los valores "NaN" con 0.

# 1) Primera forma

In [11]:
subset_nfl_data.fillna(0) # rellena con 0 los na's. Aplicaría solamente para numericas (para string object no) porque no me va a dar significancia en el analisis.

# NO FUNCIONARÍA CON STRINGS

,EPA,airEPA,yacEPA,Home_WP_pre,Away_WP_pre,Home_WP_post,Away_WP_post,Win_Prob,WPA,airWPA,yacWPA,Season
0,2.014474,0.000000,0.000000,0.485675,0.514325,0.546433,0.453567,0.485675,0.060758,0.000000,0.000000,2009
1,0.077907,-1.068169,1.146076,0.546433,0.453567,0.551088,0.448912,0.546433,0.004655,-0.032244,0.036899,2009
2,-1.402760,0.000000,0.000000,0.551088,0.448912,0.510793,0.489207,0.551088,-0.040295,0.000000,0.000000,2009
3,-1.712583,3.318841,-5.031425,0.510793,0.489207,0.461217,0.538783,0.510793,-0.049576,0.106663,-0.156239,2009
4,2.097796,0.000000,0.000000,0.461217,0.538783,0.558929,0.441071,0.461217,0.097712,0.000000,0.000000,2009


También podría ser un poco más inteligente y reemplazar los valores faltantes con cualquier valor que venga directamente después en la misma columna. (Esto tiene mucho sentido para conjuntos de datos donde las observaciones tienen algún tipo de orden lógico).

# 2) Segunda forma

In [12]:
subset_nfl_data.fillna(method='bfill', axis=0).fillna(0) # rellena con el valor de abajo. axis = 0 es para las filas. axis = 1 es para las columnas 
# el ultimo lo pone en cero porque no hay nada abajo de la ultima fila. Si tengo 2 o más seguidos que son NAn en el dataset original, pasa esto:

# valor original --> valor original
# nan --> se rellena con el anterior
# nan --> 0
# nan --> 0

# FUNCIONARÍA CON STRINGS

,EPA,airEPA,yacEPA,Home_WP_pre,Away_WP_pre,Home_WP_post,Away_WP_post,Win_Prob,WPA,airWPA,yacWPA,Season
0,2.014474,-1.068169,1.146076,0.485675,0.514325,0.546433,0.453567,0.485675,0.060758,-0.032244,0.036899,2009
1,0.077907,-1.068169,1.146076,0.546433,0.453567,0.551088,0.448912,0.546433,0.004655,-0.032244,0.036899,2009
2,-1.402760,3.318841,-5.031425,0.551088,0.448912,0.510793,0.489207,0.551088,-0.040295,0.106663,-0.156239,2009
3,-1.712583,3.318841,-5.031425,0.510793,0.489207,0.461217,0.538783,0.510793,-0.049576,0.106663,-0.156239,2009
4,2.097796,0.000000,0.000000,0.461217,0.538783,0.558929,0.441071,0.461217,0.097712,0.000000,0.000000,2009


# 3) Tercera forma

In [13]:
# haces una media de la columna y haces el relleno con eso